build_dataset 해당

In [1]:
from core.macro_classes.macro_class_dataset import MacroAData

macro_agent = MacroAData()
macro_agent.fetch_data()
macro_agent.add_features()
macro_agent.save_csv()
macro_agent.make_close_price()

[*********************100%***********************]  15 of 15 completed
C:\Users\SPOTV\IdeaProjects\capstone_project\core\macro_classes\macro_class_dataset.py:85: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=0)
C:\Users\SPOTV\IdeaProjects\capstone_project\core\macro_classes\macro_class_dataset.py:133: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(


[MacroSentimentAgent] Data shape: (1304, 90), Columns: 90
[TRACE A] add_features() for self.data:            Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X    Open_GC=F  Open_HG=F  \
Date                                                                          
2020-01-01        NaN            NaN       1.122083          NaN        NaN   
2020-01-02  61.599998      96.480003       1.121894  1518.099976     2.8165   
2020-01-03  61.180000      96.790001       1.117081  1530.099976     2.7935   
2020-01-06  63.709999      96.900002       1.116246  1580.000000     2.7780   
2020-01-07  62.910000      96.650002       1.119583  1558.300049     2.8010   
...               ...            ...            ...          ...        ...   
2024-12-24  69.559998     108.160004       1.040583  2613.000000     4.0525   
2024-12-25        NaN            NaN       1.040258          NaN        NaN   
2024-12-26  70.199997     108.169998       1.039955  2628.500000     4.0730   
2024-12-27  69.680000     108.0800

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows


In [2]:
from core.macro_classes.macro_class_dataset import MacroAData
macro_agent = MacroAData()
macro_agent.model_maker()

C:\Users\SPOTV\IdeaProjects\capstone_project\core\macro_classes\macro_class_dataset.py:164: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro_ret = macro_df[macro_features].pct_change()
C:\Users\SPOTV\IdeaProjects\capstone_project\core\macro_classes\macro_class_dataset.py:177: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_df = price_df.fillna(method='bfill')


[INFO] 병합 후 데이터 shape: (1257, 285)
Epoch 1/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - loss: 0.1484 - val_loss: 0.1307
Epoch 2/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - loss: 0.1347 - val_loss: 0.0847
Epoch 3/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.1293 - val_loss: 0.0952
Epoch 4/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1274 - val_loss: 0.1022
Epoch 5/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.1259 - val_loss: 0.0944
Epoch 6/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.1241 - val_loss: 0.1121
Epoch 7/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.1242 - val_loss: 0.0818
Epoch 8/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.1234 - val_loss: 0.0876
Epoch 9/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.1215 - val_loss: 0.0845
Epoch 10/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - loss: 0.1233 - val_loss: 0.0933
Epoch 11/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.1213 - val_loss: 0.0812
Epoch 12/60
55/55 ━━━━━━━━━━

$$$$   X = macro_sercher(agent, ticker) 역할

In [3]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료


In [4]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (44, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed

[MacroSentimentAgent] Stock data: (43, 9)
[MacroSentimentAgent] Data shape: (43, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (43, 184)
[OK] 매크로 데이터 수집 완료: (43, 185)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 183 / 스케일러 기준 피처 수: 183
[OK] 스케일링 및 시퀀스 변환 완료


In [5]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

X_seq: [[[-3.58716003e-01 -3.78179663e-01  1.05884921e+00 ... -1.15013917e-01
    3.59026218e+00  3.63362588e+00]
  [-4.34728495e-01 -4.25056313e-01  1.11257430e+00 ...  1.80760152e-02
    3.59026218e+00  3.63362588e+00]
  [-3.99951473e-01 -4.05958615e-01  1.09840824e+00 ... -1.01681348e-01
    3.64010922e+00  3.63362588e+00]
  ...
  [-4.77951267e-01 -3.78954535e-02  7.57917528e-01 ...  5.49018948e-01
    4.37608945e+00  4.15011879e+00]
  [-4.96333524e-01  2.03645824e-03  7.42651070e-01 ... -1.25707310e+00
    4.36374573e+00  4.19667214e+00]
  [-5.63403347e-01  2.42588248e-02  6.84992780e-01 ... -6.06381612e-01
    4.30134135e+00  4.23632086e+00]]]
X_scaled:     Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X  Open_GC=F  Open_HG=F  Open_QQQ  \
0   -0.448142      -0.378180       1.075549   6.229004   1.045188  3.003366   
1   -0.426779      -0.449362       1.171576   6.427028   1.090371  3.020078   
2   -0.411378      -0.371235       1.057673   6.343827   1.077688  3.072398   
3   -0.358716    

$$$$   macro_predictor(X) 역할

In [6]:
pred_prices, target = macro_sub.m_predictor(X_seq)

[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
NVDA: 마지막 종가=197.41 → 예측 종가=193.62 (예상 수익률 -1.92%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA     197.414         193.6174           -0.0192      -1.9232   

   uncertainty  confidence  
0       0.0469      21.325  

================= 예측 결과 (값) =================
{'NVDA': 193.61740833527153}


In [7]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

pred_prices: {'NVDA': 193.61740833527153}
target: Target(next_close=193.6174, uncertainty=0.046893410384655, confidence=21.325)


llm 생성 용.. macro_reviewer_draft

In [ ]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

In [ ]:
print(f"total_json: {total_json}")
print(f"opinion: {opinion}")

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [ ]:
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [ ]:
debate_agent.run_dataset()

In [ ]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


In [ ]:
opinions = debate_agent.get_opinion(0, "NVDA")

In [ ]:
print(f"opinions:{opinions}")

리뷰탈 해보기..

In [ ]:
debate_agent.get_rebuttal(1)

get_revise 해보기

In [ ]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()